In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from statistics import mean
from datetime import timedelta

import math

import warnings  
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv("~/Desktop/NCSA_genomics/prediabetic.csv")
data['DisplayTime'] = pd.to_datetime(data['DisplayTime'])
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])

In [4]:
def fullDay(data):
    dates = list()
    data = data.reset_index(drop=True)
    for i in range(0,len(data.index)):
        dates.append(data['DisplayTime'][i].date())
    data['Dates'] = dates
    end = data['Dates'].iloc[-1]
    start = data['Dates'].iloc[0]

    indexVals = data[ data['Dates'] == start ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    indexVals = data[ data['Dates'] == end ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    data = data.reset_index(drop=True)

    data.drop(['Dates'], axis=1, inplace=True)

    return data

def hourlySamples(df):
    groupkey = df['DisplayTime'].values.astype('datetime64[h]')
    result = df.groupby(groupkey).mean()
    result = result.reset_index(drop=True)
    return (result)

def subSample(data):
    data = data.reset_index(drop=True)
    data['DisplayTime'] = pd.to_datetime(data['DisplayTime'])
    data['time_gap'] = data['DisplayTime'].shift(1)-data['DisplayTime'][0]
    data['time_gap'][0] = '00:00:00'
    mods = [0,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899]
    subset = pd.DataFrame()
    for i in range(1,len(data.index)):
        seconds = data['time_gap'][i].total_seconds()
        if (seconds%900) in mods:
            subj_id = data['subjectId'][i]
            gv = data['GlucoseValue'][i]
            dt = data['DisplayTime'][i]
            temp_df = pd.DataFrame({'DisplayTime':[dt], 'GlucoseValue':[gv], 'subjectId':[subj_id]})
            subset = pd.concat([temp_df,subset],ignore_index=True)
    subset = subset.iloc[::-1]
    subset = subset.reset_index(drop=True)
    data.drop(['time_gap'], axis=1, inplace=True)
    return subset

In [5]:
def IGC(df, unit, lltr = 80, ultr = 140, a = 1.1, b = 2.0, c = 30, d = 30):
    if unit == 'mg':
        gv = df['GlucoseValue']
    elif unit == 'mmol':
        gv = 18*df['GlucoseValue']
    else:
        print('Unit should either be mg or mmol')
        return 0
    
    lower_gv = gv[gv < 90]
    upper_gv = gv[gv > 140]
    
    
    count_lower = len(lower_gv.index)
    count_upper = len(upper_gv.index)
    
    hypoglycemicIndex = np.sum(np.power((lltr - lower_gv), b)) / (count_lower*d)   
    hyperglycemicIndex = np.sum(np.power((upper_gv - ultr), a)) / (count_upper*c)
    
    if np.isnan(hypoglycemicIndex):
        hypoglycemicIndex = 0
    if np.isnan(hyperglycemicIndex):
        hyperglycemicIndex=0
    
    igc = hypoglycemicIndex + hyperglycemicIndex
    return igc, hypoglycemicIndex, hyperglycemicIndex

In [6]:
def glucoseLiabilityIndex(data, unit):
    data = hourlySamples(data)
    if unit == 'mg':
        data['GlucoseValue'] = data['GlucoseValue']/18
    gli = np.sum(np.power(data['GlucoseValue'][i] - data['GlucoseValue'][i+1],2) for i in range(0, len(data.index)-1))
    return round(gli)

In [7]:
def adrr(xx, unit):
    
    if unit == 'mg':
        f_bg = 1.509*(np.log(xx['GlucoseValue'])**1.084)-5.381
        xx['F(BG)'] = f_bg
    elif unit == 'mmol':
        f_bg = 1.509*(np.log(xx['GlucoseValue']*18)**1.084)-5.381
        xx['F(BG)'] = f_bg
    else:
        print('Unit should either be mg or mmol')
        return 0
    
    dates = []
    for i in range(len(xx.index)):
        dates.append(xx['DisplayTime'][i].date())
    xx['Date'] = dates 
    

    for Date, df in xx.groupby('Date'):
        r_BG = 0
        rl_BG = [0]
        rh_BG = [0]
        LR = 0
        HR = 0
        ADDR_daily = []
        for f_BG in df['F(BG)']:
            if f_BG < 0:
                rl_BG.append(f_BG)
            else:
                rh_BG.append(f_BG)

        LR = max(rl_BG)
        HR = max(rh_BG)
        ADDR_daily.append(LR+HR)
    
    
    return round(mean(ADDR_daily),3)
        
            

In [40]:
def modd(data):
#     data = self.subSample(data)
    data['DisplayTime'] = data['DisplayTime'].dt.round('5min') 

    times = []
    for i in range(len(data.index)):
        times.append(data['DisplayTime'][i].time())
    data['Time'] = times  


    Modd = []
    s = []
    gvDiff = 0
    for Time, df in xx.groupby('Time'):
        gvDiff = abs(df['GlucoseValue'] - df['GlucoseValue'].shift(-1))
        gvDiff = gvDiff.dropna()
        s.append(mean(gvDiff))
    
    return round(mean(s),2)

In [9]:
def congaN(df, n):
    day = df['DisplayTime'].iloc[-1]-df['DisplayTime'].iloc[0]
    day = day.round("d")
    day = day.days
    
    df = df.set_index(['DisplayTime'])
    t = str(n*3600)+'s'
    gv = df['GlucoseValue'].resample(t).first()
    
    k = len(gv)
    
    frame = pd.DataFrame()
    frame['GV'] = gv
    frame['Dt'] = frame['GV'] - frame['GV'].shift(+1)
    frame = frame.fillna(0)
    
    dBar = sum(frame['Dt']) / k
    
    s = 0
    for i in frame['Dt']:
        s += (i-dBar)**2
        
    conga = math.sqrt(s/(k-1))
    
    return round(conga/day, 3)
    

In [37]:
xx = data[data['subjectId'] == "1636-69-032"]
# xx = fullDay(xx)

In [11]:
IGC(xx, 'mg')

(1.740718384240811, 1.4662309368191722, 0.2744874474216389)

In [12]:
glucoseLiabilityIndex(xx, 'mg')

25.0

In [13]:
adrr(xx, 'mg')

3.174

In [41]:
xx

,Unnamed: 0,subjectId,DisplayTime,GlucoseValue,Time
0,1,1636-69-032,2016-01-14 00:03:00,120,00:03:00
1,2,1636-69-032,2016-01-14 00:08:00,120,00:08:00
2,3,1636-69-032,2016-01-14 00:13:00,123,00:13:00
3,4,1636-69-032,2016-01-14 00:18:00,126,00:18:00
4,5,1636-69-032,2016-01-14 00:23:00,127,00:23:00
...,...,...,...,...,...
1434,1435,1636-69-032,2016-01-18 23:37:00,136,23:37:00
1435,1436,1636-69-032,2016-01-18 23:42:00,126,23:42:00
1436,1437,1636-69-032,2016-01-18 23:47:00,132,23:47:00
1437,1438,1636-69-032,2016-01-18 23:52:00,135,23:52:00


In [42]:
modd(xx)

16.22

In [15]:
conga_1 = congaN(xx, 1)
conga_2 = congaN(xx, 2)
conga_4 = congaN(xx, 4)
print("{}\t{}\t{}".format(conga_1,conga_2,conga_4))

2.542	3.379	3.843
